Hi,
so this is the first thing that comes to my mind:

1. $n$ agents, $m$ projects
1. in iterations, add "chaos":
  1. randomly choose a project $p$ and a random cycle $C$ on a set of some $z$ agents that are not yet doing anything with this project $p$ **and at least one other project** (can be different for each voter in $C$);
  1. for each agent in this cycle, choose a superset of projects $S$, $p \in S$, to delegate to the next agent on the cycle (e.g., if we choose project $p_1$ and agents $a, b, c$, we can delegate $\{p_1,p_2\}$ from $a$ to $b$; delegate $\{p_1, p_3, p_{65}\}$ from $b$ to $c$; and delegate $\{p_1, p_6\}$ from $c$ to $a$)
  1. halt whenever you cannot find more chaos to add = there is no project $p$ which is non-delegated by $\geq 2$ agents s.t. each agent has at least one other non-delegated project.
1. the variables are budgets for the parts of the partition




In [3]:
import random, itertools, os
from collections import defaultdict
import pyomo.environ as pyo
import numpy as np

In [4]:
def create(n,m):
    voters = defaultdict(dict)
    # voters[i][S] = some voter v' to whom i delegates S
    voters_unassigned = {v:set(range(m)) for v in range(n)}
    # voters_unassigned[i] = set of projects that i does not yet assign to anyone
    projects = list(range(m))
    projects_to_assign = {p:set(range(n)) for p in range(m)}
    # project_to_assign[p] = set of voters who do not yet delegate p
    
    while True:
        random.shuffle(projects)
        for p in projects:
            found = False
            if len(projects_to_assign[p]) == 1:
                # only one voter doesn't do anything with p
                # delegate it to himself
                v = projects_to_assign[p].pop()
                voters[v][frozenset([p])] = v
                voters_unassigned[v].remove(p)
                continue
            feasible_voters = [v for v in projects_to_assign[p] if len(voters_unassigned[v]) > 1]
            if len(feasible_voters) < 2:
                # no way to put the available voters in a non-trivial cycle
                continue
            found = True
            break
        if not found:
            break
            
        #print("feasible_voters:", feasible_voters)
        z = random.randint(2,len(feasible_voters))
        C = random.sample(feasible_voters,z)
        #print("C:",C)
        for (i,v) in enumerate(C):
            s = random.randint(1,len(voters_unassigned[v])-1)
            S = set(random.sample(list((pp for pp in voters_unassigned[v] if pp != p)),s))
            #print("S:",S,"list(voters_unassigned[v]):",list(voters_unassigned[v]))
            S.add(p)
            voters[v][frozenset(S)] = C[(i+1)%z]
            voters_unassigned[v] -= S
            #print("S:",S, "v:",v,"p:",p,"s:", s)
            for pp in S:
                #print("pp:",pp,"projects_to_assign[pp]",projects_to_assign[pp])
                #print("voters[v]",voters[v])
                projects_to_assign[pp].remove(v)
    
    # Assign remaining non-assigned project to one-self (no delegation)
    for v in range(n):
        for p in list(voters_unassigned[v]):
                #v = projects_to_assign[p].pop()
                voters[v][frozenset([p])] = v
                voters_unassigned[v].remove(p)
                projects_to_assign[p].remove(v)
    
    #print("projects_to_assign:",projects_to_assign)
    #print("voters_unassigned:", voters_unassigned)
    
    return voters
        
    #random.shuffle
    #random.randint(a,b)
    

The result of the above function is a dictionary `res` indexed by the voters, such that for a voter $v$, `res[v]` is the partition of projects, e.g. `res[v][S]=d` if $v$ delegates $S \subseteq [m]$ to $d \in [n]$, and if $d=v$ then it means that $v$ is delegating this to himself. 

The optimization problem is to find an "optimal solution" $(\bar{x}_{ij})_{(i,j) \in [n] \times [m]}$ such that $\bar{x} = f(\bar{x})$ and some other point $x$ such that $F(x)^\intercal (x - \bar{x}) < 0$, which would disprove that $F$ is pseudo-monotone, which is related to stuff being solvable by "variational inequalities" approach.

$F$ is the "pseudo-gradient" of the game, and in our case ("Martin's proportionality"), $F(x) \in \mathbb{R}^{n \times m}$ is a vector which in coordinate $v,p$ is $x_{v,p} - f(x_{v,p})$.

To make this an optimization task, we search for $x$ s.t. $F(x)^\intercal x - F(x)^\intercal \bar{x})$ is minimized (and we want to find one which is negative).

## The Model

In [5]:
def pseudomono_counterexample_model(res, n,m,fix_weights=False, even_split=False):
    model = pyo.ConcreteModel()

    model.three = pyo.RangeSet(1,3)
    model.I = pyo.RangeSet(0, n-1)
    model.J = pyo.RangeSet(0, m-1)

    model.x = pyo.Var(model.three, model.I, model.J, domain=pyo.UnitInterval)
        # We have x[1] and what it to satisfy proportionality with itself
        # x[2] is x, which we will want to violate pseudo-monotonicity
        # x[3] is f(x[2]) = f(x) because those are the coefficients in the pseudogradient...
    model.d = pyo.Var(model.I, model.J, domain=pyo.UnitInterval) # defaults

    model.D = pyo.Set(initialize=list((v,S) for v in res for S in res[v])) # delegated sets
    model.DD = pyo.Set(initialize=list(((v,S,p) for v in res for S in res[v] for p in S))) # delegated sets \times projects
    model.DDD = pyo.Set(initialize=list(((i,v,S,p) for i in model.three for v in res for S in res[v] for p in S)))
    model.b = pyo.Var(model.D, domain=pyo.UnitInterval) # budget variables
    
    if not fix_weights:
        model.w = pyo.Var(model.D, domain=pyo.NonNegativeReals, bounds=(1,100)) # delegation weights variables
        #model.w = pyo.Var(model.D, domain=pyo.Integers, bounds=(1,5)) # delegation weights variables
    else:
        model.w = {(v,S): 10 for v in res for S in res[v]}
    
    def x_budget_constraint_rule(m,it,v,S):
        return sum(model.x[(it,v,j)] for j in S) == model.b[(v,S)]

    model.x_budget_constraints = pyo.Constraint(model.three, model.D, rule=x_budget_constraint_rule)

    def defaults_constraint_rule(m,v,S):
        return sum(model.d[(v,j)] for j in S) == model.b[(v,S)]
    
    def defaults_constraint_even_split_rule(m,v,S,p):
        return model.d[(v,p)] == model.b[(v,S)] / len(S)

    if even_split:
        model.defaults_constraints = pyo.Constraint(model.DD, rule=defaults_constraint_even_split_rule)
    else:
        model.defaults_constraints = pyo.Constraint(model.D, rule=defaults_constraint_rule)

    def budget_simplex_constraint_rule(m,v):
        return sum(model.b[(v,S)] for S in res[v]) == 1

    model.budget_simplex_constraints = pyo.Constraint(model.I, rule=budget_simplex_constraint_rule)
    # model.b[(v,S)] = how much budget we give delegation S
    # model.w[(v,S)] = how much weight we give the delegate


    # m is the model, always an implicit argument
    # SHOULD BE IMPLIED by budget_simplex_constraint_rule
    # def simplex_rule(m, i):
    #    return sum(m.x[(1,i,j)] for j in m.J) == 1

    # the next line creates one constraint for each member of the set model.I
    # sum of contributions of one voter over all projects is 1
    # model.simplex_constraints = pyo.Constraint(model.I, rule=simplex_rule)
    # Proportionality constraints

    def proportionality_rule(m,it1, it2,v,S,p):
        # m is model, v voter, S delegated subset, it=1 or 2 is iteration
        delegate = res[v][S]
        denominator = sum(m.d[(v,p)] + m.w[(v,S)]*m.x[(it1,delegate,p)] for p in S)
        return m.x[(it2,v,p)] == ((m.d[(v,p)] + m.w[(v,S)]*m.x[(it1,delegate,p)]) / denominator) * m.b[(v,S)]
    
    def self_prop_rule(m,v,S,p):
        return proportionality_rule(m,1,1,v,S,p)
    
    def twothree_prop_rule(m,v,S,p):
        return proportionality_rule(m,2,3,v,S,p)
    
    model.self_proportionality = pyo.Constraint(model.DD, rule=self_prop_rule)
    model.twothree_proportionality = pyo.Constraint(model.DD, rule=twothree_prop_rule)
    
    def objective_rule(m):
        obj = sum((m.x[(2,i,j)] - m.x[(3,i,j)])*(m.x[(2,i,j)] - m.x[(1,i,j)]) for i in m.I for j in m.J)
        return obj
    model.OBJ = pyo.Objective(rule=objective_rule,sense=pyo.minimize)
    
    return model

In [6]:
n=4
m=5
res = create(n,m)

In [7]:
for i in range(1000):
    res = create(n,m)
    model = pseudomono_counterexample_model(res, n,m,fix_weights=True, even_split=True)
    #os.environ["NEOS_EMAIL"] = "koutecky@iuuk.mff.cuni.cz"
    #opt = pyo.SolverFactory("knitro")
    opt = pyo.SolverFactory("ipopt")
    opt.options['max_iter']= 100000
    # solver_manager = pyo.SolverManagerFactory('neos')
    # results = solver_manager.solve(model, opt=opt,tee=True)
    %time results = opt.solve(model)
    # opt.solve(model,tee=True)
    #results.write()
    #solver_manager = pyo.SolverManagerFactory('neos')
    #results = solver_manager.solve(model, opt=opt,tee=True)
    #results.write()
    try:
        obj = sum((model.x[(2,i,j)].value - model.x[(3,i,j)].value)*
              (model.x[(2,i,j)].value - model.x[(1,i,j)].value) for i in model.I for j in model.J)
    except TypeError:
        continue
    print()
    print("*************************")
    print("obj:", obj)
    print("*************************")
    print()
    if obj < -0.1:
        print()
        print("************ FOUND IT !!! ************************")
        break

CPU times: user 25.6 ms, sys: 13.1 ms, total: 38.6 ms
Wall time: 51.8 ms

*************************
obj: 5.1671200098538444e-18
*************************

CPU times: user 16.7 ms, sys: 8.12 ms, total: 24.8 ms
Wall time: 57.5 ms

*************************
obj: 5.510388352628865e-19
*************************

    model.name="unknown";
      - termination condition: other
      - message from solver: Too few degrees of freedom (rethrown)!
CPU times: user 13 ms, sys: 11.4 ms, total: 24.3 ms
Wall time: 32.2 ms
CPU times: user 24 ms, sys: 4.83 ms, total: 28.8 ms
Wall time: 58 ms

*************************
obj: 2.178350218960119e-19
*************************

    model.name="unknown";
      - termination condition: other
      - message from solver: Too few degrees of freedom (rethrown)!
CPU times: user 20.5 ms, sys: 4.11 ms, total: 24.6 ms
Wall time: 33.1 ms
CPU times: user 23.9 ms, sys: 8.36 ms, total: 32.2 ms
Wall time: 51.7 ms

*************************
obj: 1.2068295354224822e-19
*******

In [8]:
# Weights are all 10
# Default are all evensplit
b = dict()
for v in res:
    for S in res[v]:
        b[(v,S)] = model.b[(v,S)].value
# x is fixedpoint, y is some solution
x = np.zeros((n,m))
y = np.zeros((n,m))
for i in model.I:
    for j in model.J:
        x[i,j] = model.x[(1,i,j)].value
        y[i,j] = model.x[(2,i,j)].value
counterexample = {"res": res, "x": x,"y":y,"b":b}

In [9]:
#import cloudpickle as pickle
import pickle
name="n_4_m_5_pseudomono_fixedweights_evensplit"
backup = open(name+".pickle","wb")
#model_file = open(name+"_model.pickle","wb")
pickle.dump(counterexample,backup)
backup.close()

In [10]:
res

defaultdict(dict,
            {2: {frozenset({1, 2, 3}): 1, frozenset({0, 4}): 3},
             1: {frozenset({1, 2, 3, 4}): 0, frozenset({0}): 1},
             0: {frozenset({0, 2, 3, 4}): 3, frozenset({1}): 0},
             3: {frozenset({1, 2, 3}): 2, frozenset({0, 4}): 2}})

In [11]:
b

{(2, frozenset({1, 2, 3})): 0.3372432800100791,
 (2, frozenset({0, 4})): 0.6627567199899203,
 (1, frozenset({1, 2, 3, 4})): 0.1299434965087405,
 (1, frozenset({0})): 0.8700565034912623,
 (0, frozenset({0, 2, 3, 4})): 0.00014651448648299743,
 (0, frozenset({1})): 0.9998534855130585,
 (3, frozenset({1, 2, 3})): 0.9999434604520928,
 (3, frozenset({0, 4})): 5.65395469748886e-05}

In [12]:
x

array([[2.50189631e-08, 9.99853486e-01, 7.32347795e-05, 7.32347795e-05,
        2.50189640e-08],
       [8.70056503e-01, 1.28674551e-01, 4.26112162e-04, 4.26112162e-04,
        4.16721047e-04],
       [3.31378360e-01, 2.89037732e-01, 2.41027739e-02, 2.41027739e-02,
        3.31378360e-01],
       [2.82697750e-05, 7.37244315e-01, 1.31349573e-01, 1.31349573e-01,
        2.82697751e-05]])

In [13]:
y

array([[2.68129494e-05, 9.99853486e-01, 3.55223497e-05, 4.67656809e-05,
        3.74135069e-05],
       [8.70056503e-01, 1.93061372e-06, 3.16934233e-06, 1.08790859e-01,
        2.11475374e-02],
       [3.31388147e-01, 2.05673654e-06, 3.95971543e-06, 3.37237264e-01,
        3.31368573e-01],
       [3.02249897e-05, 4.06735454e-01, 1.03801828e-01, 4.89406178e-01,
        2.63145617e-05]])